<a href="https://colab.research.google.com/github/doduck210/Instrument_Classifier/blob/main/instrument_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#for using CoLab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#for using CoLab
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

ERROR: torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


In [3]:
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import random
torch.cuda.get_device_name(0)

'Tesla T4'

In [4]:
#데이터 확인
npz=np.load('/content/gdrive/My Drive/audio/cqt.npz')
x=npz['spec']
label=npz['instr']
print("input shape : ",x.shape) 
print("label shape : ",label.shape)

input shape :  (20488, 168, 87)
label shape :  (20488, 2)


In [5]:
use_cuda = torch.cuda.is_available()
class InstrumentClassifier(nn.Module):
  def __init__(self):
    super(InstrumentClassifier,self).__init__()
    conv1=nn.Conv2d(1,8,3,1) #8@166x85
    #ReLU
    pool1=nn.MaxPool2d(2) #8@83x42
    conv2=nn.Conv2d(8,16,3,1) #16@81x40
    #ReLU
    pool2=nn.MaxPool2d(2) #16@40x20

    self.conv_module=nn.Sequential(
        conv1,
        nn.ReLU(),
        pool1,
        conv2,
        nn.ReLU(),
        pool2
    )

    fc1=nn.Linear(16*40*20,256)
    #ReLU
    fc2=nn.Linear(256,128+46)

    self.fc_module=nn.Sequential(
        fc1,
        nn.ReLU(),
        fc2
    )

    if use_cuda:
      self.conv_module = self.conv_module.cuda()
      self.fc_module = self.fc_module.cuda()
    
  def forward(self,x):
    out=self.conv_module(x)
    dim = 1
    for d in out.size()[1:]:
      dim = dim * d
    out = out.view(-1, dim)
    out = self.fc_module(out)
    return F.softmax(out,dim=1)


In [6]:
cnn=InstrumentClassifier()

In [65]:
criterion=nn.CrossEntropyLoss()
learning_rate=1e-3
optimizer=optim.Adam(cnn.parameters(),lr=learning_rate)
num_epochs=100

In [66]:
trn_loss_list=[]
val_loss_list=[]

for epoch in range(num_epochs):
  trn_loss=0.0
  for i in range(20488):#20488):
    x_torch=torch.Tensor(x[i])
    x_torch=x_torch.unsqueeze(0).unsqueeze(0)
    x_torch=x_torch.cuda()
    y_torch=torch.Tensor(label[i])
    y_torch=y_torch[0].unsqueeze(0)
    y_torch=y_torch.cuda()
    y_torch=torch.tensor(y_torch,dtype=torch.long)
    
    optimizer.zero_grad()
    model_output=cnn(x_torch)
    loss=criterion(model_output,y_torch)
    loss.backward()
    optimizer.step()
    if (i+1)%1000==0:
      print(loss)
    del loss
    del model_output

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.1689, device='cuda:0', grad_fn=<NllLossBackward

KeyboardInterrupt: ignored